# Research Track 1 - 2° assignment with jupyter integration

In [ ]:
import sys
sys.path.append('/my_ros/src/RT_assignment_2')

import jupyros as jr
import rospy

import ipywidgets as widgets
from ipywidgets import Layout

import matplotlib as mpl
import matplotlib.pyplot as plt

import math
import time
import select
import actionlib
import actionlib.msg
from std_srvs.srv import *
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Point, Pose, Twist
from sensor_msgs.msg import LaserScan
import RT_assignment_2.msg
from RT_assignment_2.msg import Pos

In [ ]:
# Create widgets
goal_wdgt = widgets.Text(placeholder='x,y', layout=Layout(width='15%'))
goal_canc_btn = widgets.Button(description='Cancel goal', button_style='danger')
nearest_obst_wdgt = widgets.FloatText(description='Nearest obstacle')
robot_pos_wdgt = widgets.Text(description='Current position', layout=Layout(width='40%'))
canc_goals_wdgt = widgets.IntText(description='Goals canceled', value=0, layout=Layout(width='15%'))
set_goals_wdgt = widgets.IntText(description='Goals set', value=0, layout=Layout(width='15%'))

In [ ]:
# Counters
canc_goals = 0
set_goals = 0

# Initialize the node
rospy.init_node('client')

# Publisher for the position of the robot
pub = rospy.Publisher('/pos', Pos, queue_size=1)

# Initialize the action client
client = actionlib.SimpleActionClient('/reaching_goal', RT_assignment_2.msg.PlanningAction)
# Wait for the action server to start
client.wait_for_server()

In [ ]:
def callback(data):
    global robot_pos_wdgt
    # Get position and linear velocity from odometry
    position = data.pose.pose.position
    linear_velocity = data.twist.twist.linear
    # Create a message of type Pos
    msg = Pos()
    # Fill the message with the position and linear velocity
    msg.x = position.x
    msg.y = position.y
    msg.vx = linear_velocity.x
    msg.vy = linear_velocity.y
    # Set current position value
    robot_pos_wdgt.value = f'{msg.x},  {msg.y}'
    # Publish the message
    pub.publish(msg)

In [ ]:
def get_nearest_obst(data):
    global nearest_obst_wdgt
    nearest_obst = min(data.ranges)
    nearest_obst_wdgt.value = nearest_obst

In [ ]:
def send_coords(wdgt):
    global set_goals
    global set_goals_wdgt
    if wdgt.value:
        set_goals += 1
        set_goals_wdgt.value = set_goals
        # Create a message of type PlanningGoal
        goal = RT_assignment_2.msg.PlanningGoal()
        # Fill the message with the goal coordinates
        goal.target_pose.pose.position.x = float(wdgt.value.split(',')[0])
        goal.target_pose.pose.position.y = float(wdgt.value.split(',')[1])
        # Send the goal to the action server
        client.send_goal(goal)

In [ ]:
def canc_goal(btn):
    global canc_goals
    global canc_goals_wdgt
    canc_goals += 1
    canc_goals_wdgt.value = canc_goals
    client.cancel_goal()

In [ ]:
# Subscriber to the odometry of the robot to get the position and velocity
odom = rospy.Subscriber('/odom', Odometry, callback)

In [ ]:
# Get nearest obstacle
obst_scan = rospy.Subscriber('/scan', LaserScan, get_nearest_obst)

In [ ]:
print("Insert the goal coordinates (x,y), then press ENTER:")

display(goal_wdgt, goal_canc_btn, nearest_obst_wdgt, robot_pos_wdgt, canc_goals_wdgt, set_goals_wdgt)

goal_wdgt.on_submit(send_coords)
goal_canc_btn.on_click(canc_goal)